### Hello

In [2]:
# pip install torch


In [3]:
import torch

d:\repos\llms\LLMs\Fundamentals\.venv\Lib\site-packages\torch\_subclasses\functional_tensor.py:279: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\torch\csrc\utils\tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


In [4]:
class MultiHeadAttention(torch.nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        # assert (d_out % num_heads == 0), 
        #     "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduce the projection dim to match desired output dim

        self.W_query = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = torch.nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = torch.nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = torch.nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape

        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # We implicitly split the matrix by adding a `num_heads` dimension
        # Unroll last dim: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim) 
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Compute scaled dot-product attention (aka self-attention) with a causal mask
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Original mask truncated to the number of tokens and converted to boolean
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use the mask to fill attention scores
        attn_scores.masked_fill_(mask_bool, -torch.inf)
        
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2) 
        
        # Combine heads, where self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

In [5]:
class GELU(torch.nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, x):
        return 0.5 * x * (1 + torch.tanh(
            torch.sqrt(torch.tensor(2.0 / torch.pi)) * 
            (x + 0.044715 * torch.pow(x, 3))
        ))

In [6]:
class Feed_Forward(torch.nn.Module):
    def __init__(self,cfg):
        super().__init__()
        self.cfg = cfg
        self.layer = torch.nn.Sequential(
            torch.nn.Linear(cfg['emb_dim'],4*cfg['emb_dim']),
            GELU(),
            torch.nn.Linear(4*cfg['emb_dim'],cfg['emb_dim'])
        )

    def forward(self,x):
        return self.layer(x)

In [7]:
class Layer_Norm(torch.nn.Module):
    def __init__(self,emb_dim):
        super().__init__()
        self.emb_dim = emb_dim
        self.eps = 6e-8
        self.scale = torch.nn.Parameter(torch.ones(emb_dim))
        self.shift = torch.nn.Parameter(torch.zeros(emb_dim))
    def forward(self,x):
        mean = x.mean(dim=-1,keepdim = True)
        var = x.var(dim = -1,keepdim = True)
        out = (x - mean)/torch.sqrt(var + self.eps)
        out = out * self.scale + self.shift

        return out

In [8]:
GPT_CONFIG_124M = {
    "vocab_size": 50257,    # Vocabulary size
    "context_length": 1024, # Context length is the length of the input sequence means no of tokens in each input sequence
    "emb_dim": 768,         # Embedding dimension
    "n_heads": 12,          # Number of attention heads
    "n_layers": 12,         # Number of layers
    "drop_rate": 0.1,       # Dropout rate
    "qkv_bias": False       # Query-Key-Value bias
}

In [ ]:
class Transformer(torch.nn.Module):
    def __init__(self,cfg):
        super().__init__()
        self.attn = MultiHeadAttention(
            d_in = cfg['emb_dim'],
            d_out = cfg['emb_dim'],#this will be the context vector dim for each token
            context_length= cfg['context_length'],#total no of tokens in the input sequence
            dropout=cfg['drop_rate'],
            num_heads= cfg['n_heads'],
            qkv_bias=cfg['qkv_bias']
        )
        self.ff = Feed_Forward(cfg)
        self.norm_1 = Layer_Norm(cfg['emb_dim'])
        self.norm_2 = Layer_Norm(cfg['emb_dim'])
        self.dropout_shortcut = torch.nn.Dropout(cfg['drop_rate'])

    def forward(self,x):
        shortcut = x
        x = self.norm_1(x)
        x = self.attn(x)
        x = self.dropout_shortcut(x)
        x = x + shortcut
        # now ff
        shortcut = x
        x = self.ff(x)
        x = self.norm_2(x)
        x = self.dropout_shortcut(x)
        x = x+shortcut

        return x
        
        
    

In [17]:
input = torch.randn(2,10,768) # batch size, num tokens, embedding dim
model = Transformer(GPT_CONFIG_124M)
output = model(input)
print(output.shape)  # Should be (2, 10, 768)

torch.Size([2, 10, 768])


In [18]:
print(output)

tensor([[[ 2.0394e+00,  7.5315e-01,  1.7883e+00,  ..., -7.1899e-01,
          -1.1229e+00,  2.5578e+00],
         [-4.5705e-01,  1.7607e+00, -2.0672e-01,  ...,  2.8193e-01,
          -8.6818e-01,  3.0153e+00],
         [-1.4981e-01,  1.9908e-03, -6.8572e-01,  ...,  2.7051e+00,
           1.5202e+00, -3.7383e-01],
         ...,
         [ 2.9835e+00,  4.2291e+00,  8.4094e-01,  ...,  2.4900e+00,
           3.3171e+00,  3.9640e-01],
         [-5.9912e-01,  1.2633e+00,  8.2014e-01,  ...,  3.7516e+00,
           2.4983e-01, -7.3212e-01],
         [-1.6793e+00,  2.1012e+00,  1.3901e+00,  ...,  3.1139e+00,
          -2.5399e+00,  1.0494e+00]],

        [[ 6.4858e-01, -9.6888e-01,  4.7311e+00,  ...,  1.2708e+00,
           2.8133e+00,  1.7985e+00],
         [ 1.1837e+00, -8.7358e-01,  2.4468e+00,  ...,  1.0566e+00,
           1.4840e+00, -9.9338e-01],
         [ 1.1871e+00,  3.5605e-01, -5.7051e-01,  ...,  1.3507e+00,
           3.3056e+00, -2.6242e+00],
         ...,
         [ 2.9649e+00, -6